In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

## Chaotic Map Function
A chaotic map generates pseudo-random sequences that are highly sensitive to initial conditions. The 2D CCM uses complex numbers to create more complex and secure sequences.

In [15]:
l = 3 + 6j
l.real, l.imag
le = 0.1*np.sin

(3.0, 6.0)

In [35]:
def CCM(x, y, a, b): # x, y -> initial conditions & a, b -> parameters
    z = x + 1j*y
    print(z.real, z.imag)
    z_new = a * np.sin(z.real) + b * np.cos(z.imag)
    return z_new.real, z_new.imag

def genSeq(len, init_x, init_y, a, b):
    seq = []
    x, y = init_x, init_y
    for _ in range(len):
        x, y = CCM(x, y, a, b)
        seq.append((x, y))
    return np.array(seq)


In [36]:
CCM(2, 10, 0.1, 0)

2.0 10.0


(0.09092974268256818, 0.0)

## Watermark Embedding Process
Frame Selection: Select frames from the video for watermarking.<br>
Transformation: Apply a transformation (e.g., DWT, SVD) to convert the frame into a domain where watermarking is more robust. <br>
Chaotic Sequence Generation: Generate a chaotic sequence using the 2D CCM with specific initial conditions and parameters.<br>
Watermark Embedding: Embed the watermark bits into the transformed coefficients using the chaotic sequence.<br>
Inverse Transformation: Apply the inverse transformation to get the watermarked frame.<br>

In [29]:
def embed_watermark(frame, watermark, chaotic_sequence):
    # Apply SVD
    U, S, Vt = np.linalg.svd(frame, full_matrices=False)
    # Embed watermark using chaotic sequence
    for i, (x, y) in enumerate(chaotic_sequence):
        S[i] += watermark[i % len(watermark)] * (x + y)
    # Reconstruct frame
    watermarked_frame = np.dot(U, np.dot(np.diag(S), Vt))
    return watermarked_frame


## Watermark Extraction Process
Frame Selection: Select the watermarked frames from the video.<br>
Transformation: Apply the same transformation as in the embedding process.<br>
Chaotic Sequence Generation: Generate the same chaotic sequence using the same initial conditions and parameters.<br>
Watermark Extraction: Extract the watermark bits from the transformed coefficients using the chaotic sequence.<br>

In [30]:
def extract_watermark(frame, chaotic_sequence, watermark_length):
    # Apply SVD
    U, S, Vt = np.linalg.svd(frame, full_matrices=False)
    # Extract watermark using chaotic sequence
    watermark = []
    for i, (x, y) in enumerate(chaotic_sequence):
        if i < watermark_length:
            watermark.append(S[i] / (x + y))
    return np.array(watermark)


## Practical Implementation
To put it all together, let's implement the complete watermarking process for a 5-second video with dimensions 128x128.

Video Processing
Load Video: Use OpenCV to read the video frames.<br>
Embed Watermark: Apply the embedding algorithm to each frame.<br>
Save Watermarked Video: Write the watermarked frames back to a video file.<br>
Extract Watermark: Verify the extraction process.<br>

In [32]:
import cv2
import numpy as np

# Load video
video_path = 'video.3gp'
cap = cv2.VideoCapture(video_path)

# Create video writer for output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (128, 128), isColor=False)

# Generate chaotic sequence
chaotic_sequence = genSeq(128, 0.1, 0.1, 1.0, 0.5)

# Define watermark (simple binary pattern for demonstration)
watermark = np.random.choice([0, 1], size=(128,))

# Process each frame
frame_count = 0
while cap.isOpened() and frame_count < 100:  # Assuming 20 fps, 100 frames for 5 seconds
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale and resize to 128x128
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (128, 128))

    # Embed watermark in frame
    watermarked_frame = embed_watermark(frame, watermark, chaotic_sequence)

    # Write watermarked frame to output video
    out.write(watermarked_frame.astype(np.uint8))

    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()

# For extraction, let's assume we process the saved watermarked video
cap = cv2.VideoCapture('output_video.mp4')

# Extract watermark from first frame for demonstration
ret, frame = cap.read()
if ret:
    # Convert to grayscale and resize to 128x128
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (128, 128))

    # Extract watermark
    extracted_watermark = extract_watermark(frame, chaotic_sequence, len(watermark))
    print("Extracted Watermark:", extracted_watermark)

cap.release()
cv2.destroyAllWindows()

extracted_watermark.shape

Extracted Watermark: [2.00465737e+04 1.95272604e+03 1.41971814e+03 7.98581053e+02
 5.50296557e+02 5.14870955e+02 4.52904893e+02 4.33094847e+02
 3.80468644e+02 3.54931900e+02 3.26092269e+02 3.09733324e+02
 2.99752546e+02 2.86743552e+02 2.78671982e+02 2.64131669e+02
 2.48760727e+02 2.44053028e+02 2.37193369e+02 2.31070234e+02
 2.25692708e+02 2.21486314e+02 2.20981595e+02 2.13708360e+02
 1.98864718e+02 1.95706430e+02 1.93406824e+02 1.91936335e+02
 1.85042539e+02 1.83574424e+02 1.77791890e+02 1.75516947e+02
 1.74282049e+02 1.71028020e+02 1.62710553e+02 1.61824404e+02
 1.59741663e+02 1.56728186e+02 1.54728160e+02 1.51490386e+02
 1.47986486e+02 1.47640140e+02 1.43731415e+02 1.41144774e+02
 1.40838634e+02 1.38062738e+02 1.37728843e+02 1.34784327e+02
 1.29825895e+02 1.28758404e+02 1.26542771e+02 1.23419586e+02
 1.20684576e+02 1.19239728e+02 1.17740261e+02 1.16262677e+02
 1.11658931e+02 1.09003593e+02 1.05614259e+02 1.03356855e+02
 1.02459368e+02 1.00437860e+02 9.94445814e+01 9.64503138e+01
 9.

(128,)